<a href="https://colab.research.google.com/github/parag-ghungrudkar/Coursera_Capstone/blob/main/Coursera_capstone_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## IBM Data Science Capstone Project
###### The Battle of neighborhood: The business object of this project is to cluster neighborhood with similar venues together. Further explanation will be detailed in this ...placeholder

###week 2:
Creating a notebook for the project. I am using google colab for the purpose.

###Week 3
## Part 1:

Creating a dataframe: placeholder that consist of three columns: PostalCode, Borough, and Neighborhood

In [1]:
! pip install geocoder #installing this library to get latitude and longitudes from the address

     |████████████████████████████████| 102kB 3.9MB/s 


In [ ]:
#importing the required libraries
import pandas as pd #library to work with dataframes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import numpy as np # to perform mathematical operations on series
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests #library to make http requests in python
from bs4 import BeautifulSoup #library to handle data from html files
import folium # map rendering library
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


In [ ]:
# importing the web page table into pandas data frame 

url ='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' # source url
Toronto_neighbourhood = pd.read_html(url, match = "Neighbourhood", flavor = 'bs4')[0] #importing the table directly into dataframe using pandas
Toronto_neighbourhood.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


###Data Cleansing

In [ ]:
Toronto_neighbourhood['Borough'].replace('Not assigned', np.NaN, inplace = True) #repalacing the 'Not Assigned' elements in Borough with NaN to drop them using methods
Toronto_neighbourhood.dropna(axis= 0, inplace=True) #using dropna to drop the rows with not assigned values
Toronto_neighbourhood.reset_index(inplace= True, drop = True) #reseting the index
Toronto_neighbourhood.head()


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [ ]:
# Checking if the neighbourhood columns have any elements that have 'Not Assigned' values

Toronto_neighbourhood['Neighbourhood'].str.count('Not assigned').sum()

0

In [ ]:
# final output for the part 1 of the assignment

Toronto_neighbourhood.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#week 3
###Part 2:

Finding latitude and Longitude coordinates of each neighbourhood

In [ ]:
# tried using following lines of code to get the coordinates of the location but due to the unrealibility of the library the code did not converge

'''
for row in Toronto_neighbourhood['Postal Code']:
  lat_lng_coords = None
  while(lat_lng_coords is None): 
    g = geocoder.google('{}, Toronto, Ontario'.format(row))
    lat_lng_coords = g.latlng
  latitude = lat_lng_coords[0]
  longitude = lat_lng_coords[1]
  lat_lon = lat_lon.append({'Latitude':latitude,'Longitude':longitude},ignore_index = True )'''


"\nfor row in Toronto_neighbourhood['Postal Code']:\n  lat_lng_coords = None\n  while(lat_lng_coords is None): \n    g = geocoder.google('{}, Toronto, Ontario'.format(row))\n    lat_lng_coords = g.latlng\n  latitude = lat_lng_coords[0]\n  longitude = lat_lng_coords[1]\n  lat_lon = lat_lon.append({'Latitude':latitude,'Longitude':longitude},ignore_index = True )"

In [ ]:
# using the URL provided in the assignment to import the csv file with latitude and longitude

url = 'https://raw.githubusercontent.com/parag-ghungrudkar/Coursera_Capstone/main/Geospatial_Coordinates.csv'
lat_lon = pd.read_csv(url) #reading the file into pandas dataframe
lat_lon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [ ]:
final_df = pd.merge(Toronto_neighbourhood,lat_lon,on = 'Postal Code') # merging the lat_lon location data with the orignal dataframe over 'Postal Code'
final_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Part 3: 
## Exploring and clustering the neighbourhood around toronto

*The initial dataframe's neighbourhood columns contains rows that have multiple neighbour for the same postal code. Following 'explode' function converts the clustered neighbourhoods into individual columns*

In [ ]:
#Defining the explode function

def explode(df, lst_cols, fill_value='', preserve_index=False):
    # make sure `lst_cols` is list-alike
    if (lst_cols is not None
        and len(lst_cols) > 0
        and not isinstance(lst_cols, (list, tuple, np.ndarray, pd.Series))):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)
    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()
    # preserve original index values    
    idx = np.repeat(df.index.values, lens)
    # create "exploded" DF
    res = (pd.DataFrame({
                col:np.repeat(df[col].values, lens)
                for col in idx_cols},
                index=idx)
             .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                            for col in lst_cols}))
    # append those rows that have empty lists
    if (lens == 0).any():
        # at least one list in cells is empty
        res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                  .fillna(fill_value))
    # revert the original index order
    res = res.sort_index()
    # reset index if requested
    if not preserve_index:        
        res = res.reset_index(drop=True)
    return res

In [ ]:
# using the explode function to list the neighbourhood with same postal code in different rows
Toronto_data= explode(final_df.assign(Neighbourhood=final_df.Neighbourhood.str.split(',')), 'Neighbourhood')
print('The intial number of neighbourhoods were{}, the neighbourhoods after {},'.format(final_df.shape[0], Toronto_data.shape[0]))

The intial number of neighbourhoods were(103, 5), the neighbourhoods after (217, 5),


In [ ]:
# I am only considering Borough that contains the string toronto in it
# filitering rows that contain the keyword 'Toronto' in the Borough column
toronto_df = Toronto_data[Toronto_data['Borough'].str.contains("Toronto")]
toronto_df.reset_index(inplace = True, drop = True)
print('total number of neighbourhoods in Toronto are {}' .format(toronto_df.shape[0]))

total number of neighbourhoods in Toronto 78


In [ ]:
toronto_df['Borough'].value_counts() 

Downtown Toronto    39
Central Toronto     18
West Toronto        13
East Toronto         8
Name: Borough, dtype: int64

In [ ]:
#using geolocator library to get coordinates for toronto city
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.6534817, -79.3839347.


In [ ]:
map_toronto = folium.Map(location =[latitude, longitude], zoom_start = 10) #locating toronto city to the map using folium library

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [ ]:
# defining credentials for foursquare api call

CLIENT_ID = '0GNGY4JRBUZ32X5431M5XKQKOP2RPGGC2HGQQDGHZNGPC2TF' # your Foursquare ID
CLIENT_SECRET = '5QUO4ZOXLCHHRZXSHT4E4JRRMYEE0TOYVWMVT0Y4RDOXXX1R' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0GNGY4JRBUZ32X5431M5XKQKOP2RPGGC2HGQQDGHZNGPC2TF
CLIENT_SECRET:5QUO4ZOXLCHHRZXSHT4E4JRRMYEE0TOYVWMVT0Y4RDOXXX1R


In [ ]:
toronto_df.loc[0, 'Neighbourhood'] #the first neighbourhood in the dataframe

'Regent Park'

In [ ]:
neighbourhood_latitude = toronto_df.loc[0, 'Latitude'] # neighbourhood latitude value
neighbourhood_longitude = toronto_df.loc[0, 'Longitude'] # neighbourhood longitude value

neighbourhood_name = toronto_df.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Regent Park are 43.6542599, -79.3606359.


In [ ]:
#creating the GET request URL. 
LIMIT = 100 
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)

In [ ]:
#Sending the GET request and examine the resutls
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6044890ce07df94bd7245857'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-54ea41ad498e9a11e9e13308-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16a941735',
         'name': 'Bakery',
         'pluralName': 'Bakeries',
         'primary': True,
         'shortName': 'Bakery'}],
       'id': '54ea41ad498e9a11e9e13308',
       'location': {'address': '362 King St E',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Trinity St',
        'distance': 143,
        'formattedAddress': ['362 King St E (Trinity St)',
         'Toronto ON M5A 1K9',
         'Canada'],
        'labeledLatLngs': [{'label': 'display',
 

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
#fromatting the joson files into a clean pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head() #nearby venues for the first neighbourhood

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980


In [ ]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

46 venues were returned by Foursquare.


In [ ]:
# defining functions to get venues for all the neighbourhoods of interest in the dataframe
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighbourhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude'])

Regent Park
 Harbourfront
Queen's Park
 Ontario Provincial Government
Garden District
 Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond
 Adelaide
 King
Dufferin
 Dovercourt Village
Harbourfront East
 Union Station
 Toronto Islands
Little Portugal
 Trinity
The Danforth West
 Riverdale
Toronto Dominion Centre
 Design Exchange
Brockton
 Parkdale Village
 Exhibition Place
India Bazaar
 The Beaches West
Commerce Court
 Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
 Forest Hill Road Park
High Park
 The Junction South
North Toronto West
 Lawrence Park
The Annex
 North Midtown
 Yorkville
Parkdale
 Roncesvalles
Davisville
University of Toronto
 Harbord
Runnymede
 Swansea
Moore Park
 Summerhill East
Kensington Market
 Chinatown
 Grange Park
Summerhill West
 Rathnelly
 South Hill
 Forest Hill SE
 Deer Park
CN Tower
 King and Spadina
 Railway Lands
 Harbourfront West
 Bathurst Quay
 South Niagara
 Island airport
R

In [ ]:
toronto_venues.shape

(3143, 7)

In [ ]:
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Regent Park,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Regent Park,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,Regent Park,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Regent Park,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [ ]:
neighbourhood_venues = toronto_venues.groupby('Neighbourhood').count() #number of venues in all neighbourhoods
neighbourhood_venues.head()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Adelaide,94,94,94,94,94,94
Bathurst Quay,15,15,15,15,15,15
Cabbagetown,45,45,45,45,45,45
Chinatown,63,63,63,63,63,63
Deer Park,16,16,16,16,16,16


In [ ]:
# I observed that some neighbourhood contained less than 10 venues. I decided to drop this venues as they are realtively empty
df_filter =  neighbourhood_venues[neighbourhood_venues['Venue'] <= 10] 
df_filter.reset_index(inplace=True)
df_filter.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Forest Hill Road Park,4,4,4,4,4,4
1,Summerhill East,3,3,3,3,3,3
2,Davisville North,8,8,8,8,8,8
3,Forest Hill North & West,4,4,4,4,4,4
4,Lawrence Park,4,4,4,4,4,4


In [ ]:
empty_neighbourhood = df_filter['Neighbourhood'].tolist() #getting a  list of empty neighbourhood 
empty_neighbourhood

[' Forest Hill Road Park',
 ' Summerhill East',
 'Davisville North',
 'Forest Hill North & West',
 'Lawrence Park',
 'Moore Park',
 'Rosedale',
 'Roselawn',
 'The Beaches']

In [ ]:
# dropping empty neighbourhood from venues dataframe
toronto_venues = toronto_venues[~toronto_venues['Neighbourhood'].isin(empty_neighbourhood)] 

#dropping empty neighbourhood from the toronto neighbourhood dataframe
toronto_df1 = toronto_df[~toronto_df['Neighbourhood'].isin(empty_neighbourhood)] 

In [ ]:
# one hot encoding of all the venues of the neighbourhood
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] =toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot =toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Regent Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Regent Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Adelaide,0.000000,0.000000,0.000000,0.0,0.000000,0.021277,0.000000,0.00,0.010638,0.010638,0.000000,0.021277,0.0000,0.0000,0.000000,0.0000,0.000000,0.031915,0.000000,0.010638,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021277,0.000000,0.010638,0.010638,0.000000,0.000000,0.010638,0.021277,0.000000,0.053191,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.031915,0.000000,0.106383,0.00000,0.000000,0.000000,0.00000,0.00,0.010638,0.000000,0.000000,0.021277,0.000000,0.021277,0.000000,0.000000,0.000000,0.000000,0.010638,0.000000,0.031915,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0

In [ ]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Adelaide----
            venue  freq
0     Coffee Shop  0.11
1            Café  0.05
2      Restaurant  0.04
3             Gym  0.03
4  Clothing Store  0.03


---- Bathurst Quay----
                 venue  freq
0      Airport Service  0.20
1       Airport Lounge  0.13
2              Airport  0.07
3                  Bar  0.07
4  Rental Car Location  0.07


---- Cabbagetown----
                venue  freq
0                Café  0.07
1         Coffee Shop  0.07
2          Restaurant  0.04
3  Chinese Restaurant  0.04
4              Market  0.04


---- Chinatown----
                           venue  freq
0                           Café  0.08
1                    Coffee Shop  0.06
2          Vietnamese Restaurant  0.05
3                            Bar  0.05
4  Vegetarian / Vegan Restaurant  0.05


---- Deer Park----
                venue  freq
0         Coffee Shop  0.12
1  Light Rail Station  0.12
2        Liquor Store  0.06
3  Athletics & Sports  0.06
4                 Pub  0.06


--

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Restaurant,Gym,Thai Restaurant,Deli / Bodega,Clothing Store,Bakery,Hotel,Cosmetics Shop
1,Bathurst Quay,Airport Service,Airport Lounge,Airport,Harbor / Marina,Coffee Shop,Rental Car Location,Sculpture Garden,Boat or Ferry,Airport Terminal,Airport Food Court
2,Cabbagetown,Coffee Shop,Italian Restaurant,Bakery,Pizza Place,Restaurant,Café,Pub,Gastropub,Liquor Store,Beer Store
3,Chinatown,Café,Coffee Shop,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bar,Mexican Restaurant,Grocery Store,Caribbean Restaurant,Park,Gaming Cafe
4,Deer Park,Light Rail Station,Coffee Shop,Sushi Restaurant,Restaurant,Pub,Supermarket,Fried Chicken Joint,Athletics & Sports,Bank,Pizza Place


In [ ]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 0, 1, 4, 3, 1, 4, 2, 3, 4, 4, 1, 0, 0, 1, 0, 1, 4, 1, 2, 0, 3,
       1, 1, 1, 1, 3, 0, 1, 1, 4, 1, 4, 1, 1, 1, 4, 1, 2, 1, 0, 1, 2, 1,
       1, 1, 4, 1, 1, 1, 4, 1, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 4, 1,
       1, 4], dtype=int32)

In [ ]:
neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Restaurant,Gym,Thai Restaurant,Deli / Bodega,Clothing Store,Bakery,Hotel,Cosmetics Shop
1,Bathurst Quay,Airport Service,Airport Lounge,Airport,Harbor / Marina,Coffee Shop,Rental Car Location,Sculpture Garden,Boat or Ferry,Airport Terminal,Airport Food Court
2,Cabbagetown,Coffee Shop,Italian Restaurant,Bakery,Pizza Place,Restaurant,Café,Pub,Gastropub,Liquor Store,Beer Store
3,Chinatown,Café,Coffee Shop,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bar,Mexican Restaurant,Grocery Store,Caribbean Restaurant,Park,Gaming Cafe
4,Deer Park,Light Rail Station,Coffee Shop,Sushi Restaurant,Restaurant,Pub,Supermarket,Fried Chicken Joint,Athletics & Sports,Bank,Pizza Place


In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df1 

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
toronto_merged.head() # check the last columns!


,Borough,Latitude,Longitude,Postal Code,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,43.654260,-79.360636,M5A,Regent Park,1,Coffee Shop,Bakery,Park,Pub,Breakfast Spot,Theater,Café,Chocolate Shop,Distribution Center,Electronics Store
3,Downtown Toronto,43.654260,-79.360636,M5A,Harbourfront,1,Coffee Shop,Bakery,Park,Pub,Breakfast Spot,Theater,Café,Chocolate Shop,Distribution Center,Electronics Store
6,Downtown Toronto,43.662301,-79.389494,M7A,Queen's Park,1,Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,Creperie,Park,Mexican Restaurant,Japanese Restaurant,Italian Restaurant,Fried Chicken Joint
7,Downtown Toronto,43.662301,-79.389494,M7A,Ontario Provincial Government,1,Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,Creperie,Park,Mexican Restaurant,Japanese Restaurant,Italian Restaurant,Fried Chicken Joint
15,Downtown Toronto,43.657162,-79.378937,M5B,Garden District,1,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Cosmetics Shop,Middle Eastern Restaurant,Italian Restaurant,Hotel,Japanese Restaurant,Movie Theater


In [ ]:
# create map of clusters
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2,toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4,toronto_merged.columns[[1] + list(range(5, toronto.shape[1]))]]